# Winery classification using Univariate / Bivariate / Multivariate Gaussian

# 1. Winery classification using the one-dimensional Gaussian

The **Wine** data set contains 178 labeled data points, each corresponding to a bottle of wine:
* The features (`x`): a 13-dimensional vector consisting of visual and chemical features for the bottle of wine
* The label (`y`): the winery from which the bottle came (1,2 or 3)

The data can be downloaded from the UCI repository (https://archive.ics.uci.edu/ml/datasets/wine).

## Load in the data set

We start by loading the packages we will need.

In [36]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal

# installing packages for interactive graphs
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

Next, we load the Wine data set and divide these into a training set of 130 points and a test set of 48 points.

In [6]:
data = np.loadtxt('wine.data.txt', delimiter=',')

# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']

Fix a particular "random" permutation of the data, and use these to effect the training / test split.

In [7]:
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
# Also split apart data and labels
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

We get four arrays:
* `trainx`: 130x13, the training points
* `trainy`: 130x1, labels of the training points
* `testx`: 48x13, the test points
* `testy`: 48x1, labels of the test points

Let's see how many training points there are from each class.

In [34]:
sum(trainy==1), sum(trainy==2), sum(trainy==3)

(43, 54, 33)

## Look at the distribution of a single feature from one of the wineries

Let's pick just one feature: 'Alcohol'. This is the first feature. Here is a *histogram* of this feature's values under class 1, along with the *Gaussian fit* to this distribution.

<img src="histogram.png">


To generate a figure like this, the following function, **density_plot** does this for any feature and label. The first line adds an interactive component that lets you choose these parameters using sliders. 

In [35]:
@interact_manual( feature=IntSlider(0,0,12), label=IntSlider(0,1,3) )
def density_plot(feature, label):
    plt.hist(trainx[trainy==label,feature], normed=True)
    
    mu = np.mean(trainx[trainy==label,feature]) # mean
    var = np.var(trainx[trainy==label,feature]) # variance
    std = np.sqrt(var) # standard deviation
    
    x_axis = np.linspace(mu - 3*std, mu + 3*std, 1000)
    plt.plot(x_axis, norm.pdf(x_axis,mu,std), 'r', lw=3)
    plt.title("Winery "+ str(label) )
    plt.xlabel(featurenames[feature], fontsize=14, color='red')
    plt.ylabel('Density', fontsize=14, color='red')
    plt.show()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2ZlYXR1cmUnLCBtYXg9MTIpLCBJbnRTbGlkZXIodmFsdWU9MSwgZGVzY3JpcHRpb249dSdsYWLigKY=


## Fit a Gaussian to each class

Let's define a function that will fit a Gaussian generative model to the three classes, restricted to just a single feature.

In [36]:
# Assumes y takes on values 1,2,3
def fit_generative_model(x,y,feature):
    k = 3 # number of classes
    mu = np.zeros(k+1) # list of means
    var = np.zeros(k+1) # list of variances
    pi = np.zeros(k+1) # list of class weights
    for label in range(1,k+1):
        indices = (y==label)
        mu[label] = np.mean(x[indices,feature])
        var[label] = np.var(x[indices,feature])
        pi[label] = float(sum(indices))/float(len(y))
    return mu, var, pi

Call this function on the feature 'alcohol'. What are the class weights?

In [37]:
feature = 0 # 'alcohol'
mu, var, pi = fit_generative_model(trainx, trainy, feature)
print pi[1:]

[0.33076923 0.41538462 0.25384615]


Next, display the Gaussian distribution for each of the three classes

In [38]:
@interact_manual( feature=IntSlider(0,0,12) )
def show_densities(feature):
    mu, var, pi = fit_generative_model(trainx, trainy, feature)
    colors = ['r', 'k', 'g']
    
    for label in range(1,4):
        m = mu[label]
        s = np.sqrt(var[label])
        x_axis = np.linspace(m - 3*s, m+3*s, 1000)
        plt.plot(x_axis, norm.pdf(x_axis,m,s), colors[label-1], label="class " + str(label))
        
    plt.xlabel(featurenames[feature], fontsize=14, color='red')
    plt.ylabel('Density', fontsize=14, color='red')
    plt.legend()
    plt.show()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2ZlYXR1cmUnLCBtYXg9MTIpLCBCdXR0b24oZGVzY3JpcHRpb249dSdSdW4gSW50ZXJhY3QnLCDigKY=


## Predict labels for the test set

How well can we predict the class (1,2,3) based just on one feature? The code below lets us find this out.

In [46]:
@interact( feature=IntSlider(0,0,12) )
def test_model(feature):
    mu, var, pi = fit_generative_model(trainx, trainy, feature)

    k = 3 # Labels 
    n_test = len(testy) # Number of test points
    score = np.zeros((n_test,k+1))
    
    for i in range(0,n_test):
        for label in range(1,k+1):
            score[i,label] = np.log(pi[label]) + norm.logpdf(testx[i,feature], mu[label], np.sqrt(var[label])) # using log is advantageous!
            
    predictions = np.argmax(score[:,1:4], axis=1) + 1
    
    errors = np.sum(predictions != testy)
    print "Test error using feature " + featurenames[feature] + ": " + str(errors) + "/" + str(n_test)
    
    return round(float(errors)/n_test, 3)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2ZlYXR1cmUnLCBtYXg9MTIpLCBPdXRwdXQoKSksIF9kb21fY2xhc3Nlcz0odSd3aWRnZXQtaW7igKY=


## Feature selection

In this notebook, we are looking at classifiers that use just one out of a possible 13 features. Choosing a subset of features is called `feature selection`. In general, this is something we would need to do based solely on the *training set*--that is, without peeking at the *test set*.

For the wine data, we compute the test error associated with each choice of feature.

In [47]:
# Test error
test_error = np.zeros(13)
for feature in range(0,13):
    test_error[feature] = test_model(feature)
test_error

Test error using feature Alcohol: 17/48
Test error using feature Malic acid: 23/48
Test error using feature Ash: 29/48
Test error using feature Alcalinity of ash: 23/48
Test error using feature Magnesium: 21/48
Test error using feature Total phenols: 16/48
Test error using feature Flavanoids: 8/48
Test error using feature Nonflavanoid phenols: 23/48
Test error using feature Proanthocyanins: 16/48
Test error using feature Color intensity: 10/48
Test error using feature Hue: 14/48
Test error using feature OD280/OD315 of diluted wines: 19/48
Test error using feature Proline: 17/48


array([0.354, 0.479, 0.604, 0.479, 0.438, 0.333, 0.167, 0.479, 0.333,
       0.208, 0.292, 0.396, 0.354])

Choosing feature `Flavanoids` results in the lowest test error.

# 2. Winery classification using the two-dimensional Gaussian

Our first generative model for Winery classification used just one feature. Now we use two features, modeling each class by a **bivariate Gaussian**.

## Fit a Gaussian to each class

We now define a function that will fit a Gaussian generative model to the three classes, restricted to a given list of features. The function returns:
* `mu`: the means of the Gaussians, one per row
* `covar`: covariance matrices of each of the Gaussians
* `pi`: list of three class weights summing to 1

In [12]:
# Fit a Gaussian to a data set using the selected features
def fit_gaussian2(x, features):
    mu = np.mean(x[:,features], axis=0)
    covar = np.cov(x[:,features], rowvar=0, bias=1)
    return mu, covar

In [110]:
# Assumes y takes on values 1,2,3
def fit_generative_model2(x, y, features):
    k = 3 # number of classes
    d = len(features) # number of features
    mu = np.zeros((k+1,d)) # list of means
    covar = np.zeros((k+1,d,d)) # list of covariance matrices
    pi = np.zeros(k+1) # list of class weights
    for label in range(1,k+1):
        indices = (y==label)
        mu[label,:], covar[label,:,:] = fit_gaussian2(x[indices,:], features)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, covar, pi

## Predict labels for the test points

This testing procedure is analogous to what was developed in the 1-d case. Let us see how well we can predict the class (1,2,3) based just on these two features.

In [112]:
# Now test the performance of a predictor based on a subset of features
@interact( f1=IntSlider(0,0,12,1), f2=IntSlider(6,0,12,1) )
def test_model2(f1, f2):
    if f1 == f2: # need f1 != f2
        print "Please choose different features for f1 and f2."
        return  
    
    features= [f1,f2]
    mu, covar, pi = fit_generative_model2(trainx, trainy, features)
    
    k = 3 # Labels 1,2,...,k
    nt = len(testy) # Number of test points
    score = np.zeros((nt,k+1))
    
    for i in range(0,nt):
        for label in range(1,k+1):
            score[i,label] = np.log(pi[label]) + \
            multivariate_normal.logpdf(testx[i,features], mean=mu[label,:], cov=covar[label,:,:])
            
    predictions = np.argmax(score[:,1:4], axis=1) + 1
    
    # Finally, tally up score
    errors = np.sum(predictions != testy)
    print "Test error using feature(s): ",
        
    for f in features:
        print "'" + featurenames[f] + "'" + " ",
    print
    print "Errors: " + str(errors) + "/" + str(nt) 
    
    return round(float(errors)/nt, 3)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2YxJywgbWF4PTEyKSwgSW50U2xpZGVyKHZhbHVlPTYsIGRlc2NyaXB0aW9uPXUnZjInLCBtYXjigKY=


Different pairs of features yield different test errors. Let us find out which pair of features achieves this minimum test error.

In [80]:
nf = len(testx[0])
z = np.zeros((nf, nf))
for i in range(0, nf):
        for j in range(0, nf):
            if i == j: # need i != j
                z[i,j] = 100 # a random high number
                while (j!=12):
                    j = j + 1
            z[i,j] = test_model2(i, j)
z[12,12] = 100
print(z)

[[1.00e+02 1.88e-01 2.50e-01 1.88e-01 2.92e-01 1.04e-01 8.30e-02 2.08e-01
  1.67e-01 1.67e-01 1.25e-01 1.04e-01 1.88e-01]
 [1.88e-01 1.00e+02 4.58e-01 3.33e-01 2.29e-01 2.50e-01 1.88e-01 4.38e-01
  2.92e-01 2.29e-01 3.33e-01 3.13e-01 1.88e-01]
 [2.50e-01 4.58e-01 1.00e+02 3.75e-01 4.79e-01 3.33e-01 2.08e-01 4.58e-01
  4.38e-01 2.71e-01 2.71e-01 2.71e-01 3.13e-01]
 [1.88e-01 3.33e-01 3.75e-01 1.00e+02 2.29e-01 2.50e-01 1.67e-01 3.96e-01
  4.17e-01 2.29e-01 1.46e-01 1.88e-01 2.71e-01]
 [2.92e-01 2.29e-01 4.79e-01 2.29e-01 1.00e+02 2.29e-01 1.46e-01 3.75e-01
  1.88e-01 2.08e-01 1.46e-01 1.46e-01 2.92e-01]
 [1.04e-01 2.50e-01 3.33e-01 2.50e-01 2.29e-01 1.00e+02 1.46e-01 2.71e-01
  3.13e-01 1.04e-01 1.67e-01 2.08e-01 1.88e-01]
 [8.30e-02 1.88e-01 2.08e-01 1.67e-01 1.46e-01 1.46e-01 1.00e+02 1.67e-01
  1.67e-01 6.30e-02 8.30e-02 1.46e-01 8.30e-02]
 [2.08e-01 4.38e-01 4.58e-01 3.96e-01 3.75e-01 2.71e-01 1.67e-01 1.00e+02
  3.75e-01 2.50e-01 2.92e-01 2.71e-01 2.29e-01]
 [1.67e-01 2.92e-01 4.38

In [106]:
ind = np.unravel_index(np.argmin(z, axis=None), z.shape)
print "'" + featurenames[ind[0]] + "'" + " " "and" + " " "'" + featurenames[ind[1]] + "'" + " " "yield the lowest test error of" + " " + str(z[ind])

'Flavanoids' and 'Color intensity' yield the lowest test error of 0.063


# 3. Winery classification with the multivariate Gaussian

Now, we do winery classification using the full set of 13 features.

## Fit a Gaussian generative model

We now define a function that fits a Gaussian generative model to the data.
For each class (`j=1,2,3`), we have:
* `pi[j]`: the class weight
* `mu[j,:]`: the mean, a 13-dimensional vector
* `covar[j,:,:]`: the 13x13 covariance matrix

This means that `pi` is a 4x1 array (Python arrays are indexed starting at zero, and we aren't using `j=0`), `mu` is a 4x13 array and `covar` is a 4x13x13 array.

In [113]:
def fit_generative_model(x,y):
    k = 3  # labels 1,2,...,k
    d = (x.shape)[1]  # number of features
    mu = np.zeros((k+1,d))
    covar = np.zeros((k+1,d,d))
    pi = np.zeros(k+1)
    for label in range(1,k+1):
        indices = (y == label)
        mu[label] = np.mean(x[indices,:], axis=0)
        covar[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [ ]:
# Fit a Gaussian generative model to the training data
mu, covar, pi = fit_generative_model(trainx,trainy)

## Use the model to make predictions on the test set

In [ ]:
# Now test the performance of a predictor based on a subset of features
def test_model(mu, covar, pi, features, testx, testy):
    
    k = 3 # Labels 
    n_test = len(testy) # Number of test points
    score = np.zeros((n_test,k+1))
    
    for i in range(0,n_test):
        for label in range(1,k+1):
            score[i,label] = np.log(pi[label]) + \
            multivariate_normal.logpdf(testx[i,features], mean=mu[label,:], cov=covar[label,:,:])
            
    predictions = np.argmax(score[:,1:4], axis=1) + 1
    
    errors = np.sum(predictions != testy)
    print "Test error using all features: " + str(errors) + "/" + str(n_test) 
    
    return round(float(errors)/n_test, 3)

In [ ]:
features = range(0, (trainx.shape)[1])
test_model(mu, covar, pi, features, testx, testy)